# Regresión Lineal Múltiple

Ya teniendo clara la regresión lineal simple, es fácil comprender la múltiple

La regresión lineal múltiple se entiende de la siguinte forma:
$$\begin{equation}
f_{\vec{w},b}(\vec{x}) = w_{1}*x_{1} + w_{2}*x_{2} + ... + w_{n}*x_{n} + b
\end{equation}
$$
teniendo en cuenta que $\vec{w}$ es un vector de pesos, cuya longitud es igual a la de cualquier vector de ejemplo. El vector de pesos sería: $\vec{w} = [w_{1}, w_{2}, ..., w_{n}]$ y sobre un ejemplo x es un vector: $\vec{x} = [x_{1}, x_{2}, ..., x_{n}]$
que pertenece a una matriz X:
$X_{m,n} = \begin{equation}
\begin{bmatrix}
x_{11} & x_{12} & ... & x_{1n}\\
x_{21} & x_{22} & ... & x_{2n}\\
. & . & . & .\\
. & . & x_{ij} & .\\
. & . & . & .\\
x_{m1} & x_{m2} & ... & x_{mn}
\end{bmatrix}
\end{equation}$

La matriz de ejemplos tendrá una dimensión de $(m,n)$, que al ser hecho el producto punto por el vector $\vec{w}$ este vector debe ser de dimensión $(n,1)$ y el resultado, que será de $(m,1)$, se les debe sumar el escalar $b$.

De manera que la derivada respecto a b sigue siendo como la de una regresión simple, pero la derivada respecto a w sería:

$$
\begin{equation}
\frac{\partial{J_{\vec{w}, b}}(\vec{w}, b)}{\partial{w}} = -\frac{2}{m}\sum_{i=1}^{m}(y - wx -b)*x_{j}^{i}
\end{equation}
$$
donde i es el vector ejemplo i-ésimo y j es el dato en la posición j-ésima del vector $x^{i}$, es decir, $x_{j}^{i}$ representa un escalar, y tiene todo el sentido porque vamos a intentar calcular la derivada.

La derivada respecto al sesgo $b$ es:
$$
\begin{equation}
\frac{\partial{J_{\vec{w}, b}}(\vec{w}, b)}{\partial{b}} = -\frac{2}{m}\sum_{i=1}^{m}(y - wx -b)
\end{equation}
$$

In [1]:
import ipynb